In [1]:
import pandas as pd
import numpy as np 
import pickle

In [2]:
df = pickle.load( open( "../data/all_fed_speeches", "rb" ) )
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 733 entries, 0 to 734
Data columns (total 5 columns):
date       733 non-null datetime64[ns]
speaker    733 non-null object
title      733 non-null object
link       733 non-null object
text       733 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 34.4+ KB


In [4]:
# initial stab at tokenizing
one_doc = df['text'].iloc[0]

In [5]:
testing = one_doc.split()

5190

In [8]:
''' FROM THE LECTURE 
vocab_set = set()
for doc in tokenized_documents:
    vocab_set.update(set(doc))
vocabulary = sorted(vocab_set)

X = np.zeros(shape=(len(corpus), len(vocabulary)))

for i, doc in enumerate(tokenized_documents):
    for word in doc:
        j = vocabulary.index(word)
        X[i,j] += 1



'''

NameError: name 'tokenized_documents' is not defined

In [23]:
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import string

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davidsmith/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davidsmith/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davidsmith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
corpus = df['text']

In [14]:
# Step 1 split each doc into a list of tokens
tokenized_docs = [doc.split() for doc in corpus]


In [20]:
# Step 2: lowercase everything 
t_docs_lower = [[word.lower() for word in doc]
               for doc in tokenized_docs]

In [42]:
# Step 3: remove punctuation
punct = set(string.punctuation)


In [44]:
def remove_symbols(word, symbol_set):
    return ''.join(char for char in word if char not in symbol_set)

In [45]:
cleaned_docs = [[remove_symbols(word, punct) for word in doc] for doc in t_docs_lower]

In [46]:
# Step 4: Remove stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
#print(stop_words)

{'until', 'because', "wasn't", 'hadn', "you'd", 'through', "won't", 'was', 'further', 'down', 'are', 'as', "don't", 'don', 'shouldn', "aren't", 'did', 'more', "she's", 'wouldn', 'against', 'does', "weren't", 'whom', 'hers', 'with', 'between', "didn't", 'once', "hasn't", 'yourself', 'into', 'about', 'wasn', "doesn't", 'other', 'there', 'just', 'both', 'its', 'now', "you're", 'before', 'to', 'o', 'only', 'same', 'then', 'all', 'and', 'ain', 'some', 'where', 'but', 'of', "that'll", "should've", 'being', 'in', 'which', 'should', 'couldn', 'each', 'itself', 'out', "shouldn't", 'from', 'at', 'herself', 'be', 'your', 'she', 'd', 'few', 'my', 'own', 'he', 'his', 'why', 'weren', 'doing', 'will', 'ma', 'yourselves', 'i', 'most', 'no', 'aren', 't', 'do', 're', 'her', 'shan', 'so', 'here', "isn't", 'am', 'me', 'mightn', 'has', 'nor', 'above', 'yours', 'him', 'y', 'those', 'a', 'theirs', "couldn't", 'too', 'or', 'm', 'if', 've', 'such', 'our', 'over', 'very', "mustn't", 'not', 'haven', 'ours', 'aft

In [47]:
docs_no_stops = [[word for word in doc if word not in stop_words] for doc in cleaned_docs]

In [49]:
print(type(docs_no_stops))

<class 'list'>


In [50]:
# Step 5: Stemming and Lemmatization
lemmer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [51]:
docs_lemmatized = [[lemmer.lemmatize(word) for word in doc] for doc in docs_no_stops]

In [56]:
docs_stemmed = [[stemmer.stem(word) for word in doc] for doc in docs_lemmatized]

In [62]:
def our_count_vectorizer(docs):
    '''
    Args:
        docs (list of lists of strings): corpus
    Returns:
        X_count (numpy array): count vectors
        vocab (list of strings): alphabetical list 
                                 of unique words
    '''
    vocab_set = set()
    for doc in docs:
        vocab_set.update(doc)

    vocab = sorted(vocab_set)

    X_count = np.zeros(shape=(len(docs), len(vocab)))

    for i, doc in enumerate(docs):
        for word in doc:
            j = vocab.index(word)
            X_count[i,j] += 1
    return X_count, vocab  

In [64]:
X_count, vocab = our_count_vectorizer(docs_stemmed)

In [ ]:
count_vectors = pd.DataFrame(data = X_count, columns = vocab)

In [37]:
count_vectors.shape


5190

## Term Frequency Vectors

In [ ]:
X_tf = X_count / X_count.sum(axis=1, keepdims=True)

In [ ]:
tf_vectors = pd.DataFrame(data = X_tf, columns=vocab)
tf_vectors.round(2)

## Term Frequency * Inverse Doc Frequency Vectors

In [ ]:
doc_count = np.zeros(shape=(len(vocab),))

for i,word in enumerate(vocab):
    for doc in docs_cleaned:
        if word in doc:
            doc_count[i] += 1

doc_freq = doc_count/len(corpus)
            
doc_freq_series = pd.Series(data=doc_freq, index=vocab)
doc_freq_series.round(2)

In [ ]:
inverse_doc_freq = np.log(1/doc_freq_series)

In [ ]:
idf = np.log(1/doc_freq)

In [ ]:
tf_idf = pd.DataFrame(data=X_tf*idf, columns=vocab)

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
#pairwise_dist = squareform(pdist(poem_vec, metric='cosine'))


# AFTERNOON LECTURE